<a href="https://colab.research.google.com/github/NdanyuzweP/A-CHAT-BOT/blob/main/A_Chat_BOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers tensorflow datasets
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
import pandas as pd
import numpy as np
import pandas as pd
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.6 MB/s eta 0:00:00


In [3]:
# Load dataset
file_path = "/content/medicaldataset.csv"
df = pd.read_csv(file_path)

# Display the first few rows
print(df.head())

# Check column names
print(df.columns)

                                 question  \
0                What is (are) Glaucoma ?   
1                  What causes Glaucoma ?   
2     What are the symptoms of Glaucoma ?   
3  What are the treatments for Glaucoma ?   
4                What is (are) Glaucoma ?   

                                              answer           source  \
0  Glaucoma is a group of diseases that can damag...  NIHSeniorHealth   
1  Nearly 2.7 million people have glaucoma, a lea...  NIHSeniorHealth   
2  Symptoms of Glaucoma  Glaucoma can develop in ...  NIHSeniorHealth   
3  Although open-angle glaucoma cannot be cured, ...  NIHSeniorHealth   
4  Glaucoma is a group of diseases that can damag...  NIHSeniorHealth   

  focus_area  
0   Glaucoma  
1   Glaucoma  
2   Glaucoma  
3   Glaucoma  
4   Glaucoma  
Index(['question', 'answer', 'source', 'focus_area'], dtype='object')


In [4]:
# Keep only relevant columns
df = df[['question', 'answer']]

# Combine question and answer into a conversational format
df['text'] = df['question'] + " " + df['answer']

# Check the cleaned dataset
print(df.head())

# Create the directory if it doesn't exist
os.makedirs("/mnt/data", exist_ok=True)  # This line should be moved before saving the file

# Save processed data for training
df['text'].to_csv("/mnt/data/processed_dataset.csv", index=False)

<ipython-input-4-fe1d34cb5803>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['question'] + " " + df['answer']


                                 question  \
0                What is (are) Glaucoma ?   
1                  What causes Glaucoma ?   
2     What are the symptoms of Glaucoma ?   
3  What are the treatments for Glaucoma ?   
4                What is (are) Glaucoma ?   

                                              answer  \
0  Glaucoma is a group of diseases that can damag...   
1  Nearly 2.7 million people have glaucoma, a lea...   
2  Symptoms of Glaucoma  Glaucoma can develop in ...   
3  Although open-angle glaucoma cannot be cured, ...   
4  Glaucoma is a group of diseases that can damag...   

                                                text  
0  What is (are) Glaucoma ? Glaucoma is a group o...  
1  What causes Glaucoma ? Nearly 2.7 million peop...  
2  What are the symptoms of Glaucoma ? Symptoms o...  
3  What are the treatments for Glaucoma ? Althoug...  
4  What is (are) Glaucoma ? Glaucoma is a group o...  


In [5]:
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
import tensorflow as tf

# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Add special tokens if missing
tokenizer.pad_token = tokenizer.eos_token

# Load GPT-2 model
model = TFGPT2LMHeadModel.from_pretrained("gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [6]:
# Load processed dataset
file_path = "/mnt/data/processed_dataset.csv"
df = pd.read_csv(file_path)

# Ensure 'text' column contains strings, not lists
df['text'] = df['text'].astype(str) # convert to string type

# Tokenize the text
inputs = tokenizer(list(df["text"]), max_length=512, truncation=True, padding="max_length", return_tensors="tf")

# Create TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((inputs["input_ids"], inputs["attention_mask"])).batch(8)

In [16]:
# Ensure 'text' column contains strings, not lists
df['text'] = df['text'].astype(str)  # Convert to string type

# Tokenize the text
inputs = tokenizer(
    list(df["text"]),
    max_length=512,
    truncation=True,
    padding="max_length",
    return_tensors="tf"
)

# Create TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((inputs["input_ids"], inputs["attention_mask"]))

# Function to map inputs and labels (labels = input_ids for GPT-2)
def map_to_dict(input_ids, attention_mask):
    return {"input_ids": input_ids, "attention_mask": attention_mask}, input_ids  # Labels = input_ids

train_dataset = train_dataset.map(map_to_dict)  # Apply mapping
train_dataset = train_dataset.batch(2)  # Reduce memory usage


In [9]:
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')  # Use float16 instead of float32


In [10]:
train_dataset = train_dataset.batch(2)  # Reduce memory usage

In [12]:
# Load processed dataset
file_path = "/mnt/data/processed_dataset.csv"
df = pd.read_csv(file_path)

# Ensure 'text' column contains strings, not lists
df['text'] = df['text'].astype(str) # convert to string type

# Tokenize the text
inputs = tokenizer(list(df["text"]), max_length=512, truncation=True, padding="max_length", return_tensors="tf")

# Create TensorFlow dataset
# Removed the .batch(8) here as it's redundant and causes the issue
train_dataset = tf.data.Dataset.from_tensor_slices((inputs["input_ids"], inputs["attention_mask"]))

# Apply batching with the desired batch size (2 in this case)
train_dataset = train_dataset.batch(2)  # Reduce memory usage

In [17]:
from transformers import TFGPT2LMHeadModel, AdamWeightDecay

# Load GPT-2 model
model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Define optimizer
optimizer = AdamWeightDecay(learning_rate=5e-5, weight_decay_rate=0.01)

# Define loss function (GPT-2 does NOT include loss by default)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile model
model.compile(optimizer=optimizer, loss=loss_fn)


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [18]:
# Train the model
model.fit(train_dataset, epochs=1)

# Save the fine-tuned model
model.save_pretrained("/mnt/data/fine_tuned_gpt2")
tokenizer.save_pretrained("/mnt/data/fine_tuned_gpt2")

print("Fine-tuning complete! Model saved.")

8206/8206 [==============================] - 3194s 387ms/step - loss: 0.0122
Fine-tuning complete! Model saved.


In [21]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

# Load fine-tuned model
model_path = "/mnt/data/fine_tuned_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = TFGPT2LMHeadModel.from_pretrained(model_path)

# Function to generate responses
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="tf")
    output = model.generate(**inputs, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /mnt/data/fine_tuned_gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [24]:
# Test the chatbot
user_input = "What are the symptoms of Glaucoma?"
response = generate_response(user_input)
print("Chatbot:", response)

Chatbot: What are the symptoms of Glaucoma???????????????????????????????????????????????????????????????????????????????????????????


In [25]:
!pip install fastapi uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.6 MB/s eta 0:00:00


In [26]:
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model
model_path = "/mnt/data/fine_tuned_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = TFGPT2LMHeadModel.from_pretrained(model_path)

# Initialize FastAPI
app = FastAPI()

# Request Body Model
class ChatRequest(BaseModel):
    prompt: str

# Function to generate chatbot response
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="tf")
    output = model.generate(**inputs, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# API Endpoint
@app.post("/chat")
def chat(request: ChatRequest):
    response = generate_response(request.prompt)
    return {"response": response}


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /mnt/data/fine_tuned_gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [31]:
%%writefile api.py
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model
model_path = "/mnt/data/fine_tuned_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = TFGPT2LMHeadModel.from_pretrained(model_path)

# Initialize FastAPI
app = FastAPI()

# Request Body Model
class ChatRequest(BaseModel):
    prompt: str

# Function to generate chatbot response
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="tf")
    output = model.generate(**inputs, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# API Endpoint
@app.post("/chat")
def chat(request: ChatRequest):
    response = generate_response(request.prompt)
    return {"response": response}


Writing api.py


In [30]:
import os
print(os.listdir())


['.config', 'medicaldataset.csv', 'sample_data']


In [ ]:
!uvicorn api:app --host 0.0.0.0 --port=8000 --reload

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [24514] using StatReload
2025-02-24 10:04:17.725378: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740391457.745847   24516 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740391457.752190   24516 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-24 10:04:22.135604: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:174039146